In this project, I will be find the correaltion between varibles (i.e. does a higher `budget` correlate to greater `gross`) of a Movies dataset. 

**Dataset**:
This data set was found and downleaded from [Kaggle](https://www.kaggle.com/danielgrijalvas/movies/version/2) where the user danielgrijalva scraped 4 deacades worth of movie data (7668 movies total) off of IMDb. The columns are as follows...

- *budget* : the budget of a movie. Some movies don't have this, so it appears as NaN
- *company* : the production company
- *country* : country of origin
- *director* : the director
- *genre* : main genre of the movie.
- *gross* : revenue of the movie
- *name* : name of the movie
- *rating* : rating of the movie (R, PG, etc.)
- *released* : release date
- *runtime* : duration of the movie
- *score* : IMDb user rating
- *votes* : number of user votes
- *star* : main actor/actress
- *writer* : writer of the movie
- *year* : year of release

In [1]:
# Importing libraries
import numpy as np
import pandas as pd 
import seaborn as sns
import datetime

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure 

%matplotlib inline 
matplotlib.rcParams['figure.figsize'] = (12, 8) # sets the dim of the figure outputs

In [2]:
# Now to upload the movies dataset
movies_df = pd.read_csv('movies_dataset/movies.csv')
movies_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,NaN,90.0
7664,Dream Round,NaN,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN
7666,It's Just Us,NaN,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,NaN,120.0


## I. Cleaning the data

### i. Date Format
First thing I want to do is change `movies_df.released` to Date formate YYYY-MM-DD (the kaggle descrption said it would be in YYYY-MM-DD fomrmat, but it apparently isnt. But its a good change to practice). 

The `movies_df.released` column has the date in %B %D, %Y format, followed by a '(release country)'. So I will split `movies_df.released` into two columns, `release_date` and `release_country` and replace `movies_df.released` with `movies_df.release_date`. 

In [3]:
movies_df['released'] = movies_df.released.str.split('(', expand=True)[0]

movies_df.released

0          June 13, 1980 
1           July 2, 1980 
2          June 20, 1980 
3           July 2, 1980 
4          July 25, 1980 
              ...        
7663    October 23, 2020 
7664    February 7, 2020 
7665      April 27, 2020 
7666     October 1, 2020 
7667     August 19, 2020 
Name: released, Length: 7668, dtype: object

Now to format from `%B %d, %Y ` to `%Y-%m-%d`

In [4]:
movies_df['released'] = pd.to_datetime(movies_df.released)

movies_df.released

0      1980-06-13
1      1980-07-02
2      1980-06-20
3      1980-07-02
4      1980-07-25
          ...    
7663   2020-10-23
7664   2020-02-07
7665   2020-04-27
7666   2020-10-01
7667   2020-08-19
Name: released, Length: 7668, dtype: datetime64[ns]

### ii. Missing Values
First, I want to check for missing values. 
If we run the command `movies_df.info()`, we would see that there is a lot of missing data. 

In [5]:
# running the following for-loop wil give us a count of missing/NaN values in each column
for col in movies_df.columns:
    missing_val_count = movies_df[col].isnull().sum()
    print(f"`{col}` has {missing_val_count} missing values")


# looking into NaN of specific columns
movies_df[movies_df.runtime.isnull()]

`name` has 0 missing values
`rating` has 77 missing values
`genre` has 0 missing values
`year` has 0 missing values
`released` has 2 missing values
`score` has 3 missing values
`votes` has 3 missing values
`director` has 0 missing values
`writer` has 3 missing values
`star` has 1 missing values
`country` has 3 missing values
`budget` has 2171 missing values
`gross` has 189 missing values
`company` has 17 missing values
`runtime` has 4 missing values


,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
5728,Saw: The Final Chapter,R,Crime,2010,NaT,5.6,93000.0,Kevin Greutert,Patrick Melton,Tobin Bell,NaN,NaN,NaN,NaN,NaN
5730,The Wolfman,R,Drama,2010,NaT,5.8,104000.0,Joe Johnston,Andrew Kevin Walker,Benicio Del Toro,NaN,NaN,NaN,NaN,NaN
6195,One for the Money,PG-13,Action,2012,2012-01-27,5.3,41000.0,Julie Anne Robinson,Stacy Sherman,Katherine Heigl,United States,40000000.0,38084162.0,Lakeshore Entertainment,NaN
7665,Saving Mbango,NaN,Drama,2020,2020-04-27,5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN


Some of these missing values and be inputted manually. For example, there are 4 missing runtimes (`movies_df[movies_df.released.isnull()]`) and 2 missing dates (`movies_df[movies_df.runtime.isnull()]`) which can easily be found online and replaced. 

On the otherhand, some values in columns (like `movies_df.score`) have NaN/missing because such data isnt availiable on IMDb where the data is sourced (i.e. some movies dont have user scores). Therefore, in cases like these, if I would remove rows with NaN/missing values if correlation analysis is performed on such columns. 

In [6]:
movies_df.released[[5728, 5730]] = [datetime.date(2010, 10, 29), datetime.date(2010, 2, 12)]

print(f"Now there are {movies_df.released.isnull().sum()} missing/null values for `movies_df.released`")

Now there are 0 missing/null values for `movies_df.released`


<ipython-input-6-c74f82666de2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.released[[5728, 5730]] = [datetime.date(2010, 10, 29), datetime.date(2010, 2, 12)]


In [7]:
movies_df.runtime[[5728, 5730, 6195, 7665]] = [90.0, 125.0, 91.0, 110.0]

print(f"Now there are {movies_df.runtime.isnull().sum()} missing/null values for `movies_df.runtime`")

Now there are 0 missing/null values for `movies_df.runtime`


<ipython-input-7-6a2ec20a32d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.runtime[[5728, 5730, 6195, 7665]] = [90.0, 125.0, 91.0, 110.0]


### iii. Data types

Let's look at the data types of each row...

In [8]:
movies_df.dtypes

name         object
rating       object
genre        object
year          int64
released     object
score       float64
votes       float64
director     object
writer       object
star         object
country      object
budget      float64
gross       float64
company      object
runtime     float64
dtype: object

If I wanted to change the data type of `movies_df.budget`and `movies_df.gross` from float64 to int64 (for cosmetic reasons only), I would run the following...
```
movies_df['budget'] = movies_df['budget'].astype('int64')
movies_df['gross'] = movies_df['gross'].astype('int64')
```
But given that there are many NaN variables in those columns, I cannot bc "Cannot convert non-finite values (NA or inf) to integer
". 

### iv. More on Dates

As can be seen from the query below, the year in `movies_df.year` and the year in `movies_df.released` dont match for $1324$ rows. 

In [9]:
movies_df[movies_df.year != pd.DatetimeIndex(movies_df.released).year]

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
8,Superman II,PG,Action,1980,1981-06-19 00:00:00,6.8,101000.0,Richard Lester,Jerry Siegel,Gene Hackman,United States,54000000.0,108185706.0,Dovemead Films,127.0
11,The Gods Must Be Crazy,PG,Adventure,1980,1984-10-26 00:00:00,7.3,54000.0,Jamie Uys,Jamie Uys,N!xau,South Africa,5000000.0,30031783.0,C.A.T. Films,109.0
21,Heaven's Gate,R,Adventure,1980,1981-04-24 00:00:00,6.8,14000.0,Michael Cimino,Michael Cimino,Kris Kristofferson,United States,44000000.0,3484523.0,Partisan Productions,219.0
33,Cattle Annie and Little Britches,PG,Drama,1980,1981-04-24 00:00:00,6.1,604.0,Lamont Johnson,David Eyre,Scott Glenn,United States,5100000.0,534816.0,Cattle Annie Productions,97.0
40,The Watcher in the Woods,PG,Family,1980,1981-10-09 00:00:00,6.3,5700.0,John Hough,Brian Clemens,Bette Davis,United States,NaN,5000000.0,Walt Disney Productions,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7525,Weathering with You,PG-13,Animation,2019,2020-01-17 00:00:00,7.5,28000.0,Makoto Shinkai,Makoto Shinkai,Kotaro Daigo,Japan,11100000.0,193457467.0,"""Weathering With You"" Film Partners",112.0
7580,Run with the Hunted,Not Rated,Crime,2019,2020-06-26 00:00:00,5.2,735.0,John Swab,John Swab,Ron Perlman,United States,NaN,682.0,Roxwell Films,93.0
7584,"Faith, Hope & Love",PG,Comedy,2019,2020-02-04 00:00:00,6.2,719.0,J.J. Englert,Robert Krantz,Peta Murgatroyd,United States,NaN,210091.0,Ellinas Multimedia,106.0
7604,Mine 9,Not Rated,Drama,2019,2020-05-19 00:00:00,6.4,4400.0,Eddie Mensore,Eddie Mensore,Terry Serpico,United States,350000.0,226421.0,Emphatic Films,83.0


In [10]:
# Creating the correct `year` column
movies_df['year_correct'] = pd.DatetimeIndex(movies_df.released).year

### v. Finding & Dropping Duplicates

Using the `.duplicated()` method, we can see if there are any duplicate values within columns and/or duplicate rows. 

Applying this method to a column, for example `movies_df.name`, we can see all the movies that were remade (using the code below)
```
> movies_df.loc[movies_df['name'].duplicated(), :]
```

In [30]:
num_duplicates = movies_df.duplicated().sum()

print(f"There are {num_duplicates} duplicate rows")

There are 0 duplicate rows


## II. Correlations

What I want to figure out is what correlates to higher gross revenue and better scores. 

I hypothesis that budget will have a great correlation to gross revenue but I dont think that budget alwasy determines the quailty as there are low budget films that shine brighter than high budget films (i.e. the blair witch project). Therefore, I believe that director and writers have a high correlation to better scores. 

Other questions: 
- Which comapnies/writers/directors/stars push out better scores/greater gross
    - Group by comapnies/writers/directors/stars and sum up/average out scores/gross revenue 

In [34]:
movies_df[movies_df['name'] == 'The Blair Witch Project']

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,year_correct
3459,The Blair Witch Project,R,Horror,1999,1999-07-30 00:00:00,6.5,239000.0,Daniel Myrick,Daniel Myrick,Heather Donahue,United States,60000.0,248639099.0,Haxan Films,81.0,1999
